# MPC Submission Status

#### This tutorial provides information on how to use the Minor Planet Center's Submission Status API. 

The Minor Planet Center's `Submission Status` service will return information on the status of an MPC data submission. 

This can be useful if you previously submitted observations to the MPC, received a `Submission-ID`, and would now like to understand whether that submission has been ingested into the MPC's system, or whether it has been rejected (and if so, why).
 - Further information on how to perform the precursor step of submitting observations to the MPC can be found in the following companion tutorial:
    https://docs.minorplanetcenter.net/tutorials/notebooks/mpc_tutorial_api_submission_submission.html

The Submission status API is a REST endpoint, therefore you can use your language of choice to send GET requests to the following URL:

    https://data.minorplanetcenter.net/api/submission-status

In the examples below we use python code to query the api. 

Further information and documentation concerning the `Submission Status` service and its associated api can be found at: 
   https://minorplanetcenter.net/mpcops/documentation/submission-status-api/



# Import Packages
Here we import the standard python packages need to call the api & interpret the returned data 

In [1]:
import requests
import json

# Submission Status Query 

Your request must provide a JSON object payload containing the `submission_id` of interest. 

E.g. `{"submission_id": "2026-01-01T00:05:07.453_0000BhCE"}`


### Querying a successfully accepted submission

In the following example we see that the submission was *accepted*. 

We see :
 - `"accepted": true`, indicating that the submission was accepted
 - `"pipeline_entry_time": "2026-01-01T00:06:00.696565+00:00"`, indicating that pipeline orbit processing (e.g. orbit-fitting) started approximately 53 seconds after the time of the initial submission-ID. 

In [2]:
# Call the api using the `requests` package 
response = requests.get(
    "https://data.minorplanetcenter.net/api/submission-status", 
    json= {"submission_id": "2026-01-01T00:05:07.453_0000BhCE"}
)

# Print the response, making use of the `json` package
sedna_json = response.json()
print(json.dumps(sedna_json, indent=4))


{
    "accepted": true,
    "pipeline_entry_time": "2026-01-01T00:06:00.696565+00:00",
    "fault_events": []
}


### Querying a successfully accepted submission with warnings (that did *not* prevent acceptance) 

In the following two examples we see that the submission was *accepted*, but there is some warning content in the `fault_events` field. 


In [3]:
# Loop over two examples of accepted submission with warnings that did *not* prevent acceptance
for submissionID in ["2025-11-12T18:44:08.601_0000Ba84", "2025-11-13T11:23:47.333_0000BaBf"]:
    
    print('-'*55, f"\nQuerying {submissionID=}")
    # Call the api using the `requests` package 
    response = requests.get(
        "https://data.minorplanetcenter.net/api/submission-status", 
        json= {"submission_id": "2025-11-12T18:44:08.601_0000Ba84"}
    )
    
    # Print the response, making use of the `json` package
    sedna_json = response.json()
    print(json.dumps(sedna_json, indent=4))


------------------------------------------------------- 
Querying submissionID='2025-11-12T18:44:08.601_0000Ba84'
{
    "accepted": true,
    "pipeline_entry_time": "2025-11-12T19:13:24.148269+00:00",
    "fault_events": [
        {
            "message": "warning: name `WFST NEO Survey Group of PMO and USTC.` in header CON does not match MPC format",
            "phase": 2,
            "failure_code": 10
        },
        {
            "message": "warning: postprocessed name `WFST NEO Survey Group of PMO and USTC.` did not match regex `^([A-Z]\\.[- ]){0,6}[A-Z]\\. [-.'A-Za-z ]+$`",
            "phase": 2,
            "failure_code": 10
        }
    ]
}
------------------------------------------------------- 
Querying submissionID='2025-11-13T11:23:47.333_0000BaBf'
{
    "accepted": true,
    "pipeline_entry_time": "2025-11-12T19:13:24.148269+00:00",
    "fault_events": [
        {
            "message": "warning: name `WFST NEO Survey Group of PMO and USTC.` in header CON does not m

### Querying a failed submission with errors

In the following example we see that the submission was **not** accepted, with the `fault_events` field providing information on the failure reason(s). 


In [4]:
# Call the api using the `requests` package 
response = requests.get(
    "https://data.minorplanetcenter.net/api/submission-status", 
    json= {"submission_id": "2025-11-12T21:11:49.579_0000Ba8V"}
)

# Print the response, making use of the `json` package
sedna_json = response.json()
print(json.dumps(sedna_json, indent=4))

{
    "accepted": false,
    "pipeline_entry_time": null,
    "fault_events": [
        {
            "message": "exact duplicate of submission 2025-11-12T21:09:16.359_0000Ba8U",
            "phase": 2,
            "failure_code": 5
        },
        {
            "message": "ingest failed",
            "phase": 2,
            "failure_code": 7
        }
    ]
}


### Querying a non-existent submission

Here we demonstrate that if you try to query a submission-id that does **not** exist in the MPC system, the api will return a `404` error.

In [5]:
# Call the api using the `requests` package 
response = requests.get(
    "https://data.minorplanetcenter.net/api/submission-status", 
    json= {"submission_id": "2022-02-02T22:22:22.222_0000AaCC"}
)

# Print the response status_code
if response.status_code == 404: 
    print(f"The response status code was 404, indicating that the submission id was not found")

The response status code was 404, indicating that the submission id was not found


# Using the `mpc_api` Package

The [`mpc_api`](https://github.com/Smithsonian/mpc-public/tree/main/mpc_api) Python package wraps all MPC API calls into a single `MPCClient` class.

```bash
pip install mpc-api
```

In [ ]:
from mpc_api import MPCClient

mpc = MPCClient()

# Check submission status
status = mpc.get_submission_status("2026-01-01T00:05:07.453_0000BhCE")
print(json.dumps(status, indent=4))